# Prosper Loan Data Analysis

# Deepak Sithu

## Table of Contents

* <a href="#overview">Overview</a>
* <a href="#prelim">Preliminary Wrangling</a>
* <a href="#initial">Initial Exploration</a>
* <a href="#univar">Univariate Exploration</a>
* <a href="#bivar">Bivariate Exploration</a>
* <a href="#multivar">Multivariate Exploration</a>

In [8]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<a id="overview"></a>
## Overview

> talk about the project and the dataset here XXXXX

<a id="prelim"></a>
## Preliminary Wrangling

In [9]:
# load in dataset and do head, info, shape, describe

<a id="initial"></a>
## Initial Exploration

### What is the structure of the dataset?

> asdf

### What are the main features of interest in the dataset?

> asdf

### What features in the dataset do you think will help support your investigation into the features of interest?

> asdf

<a id="univar"></a>
## Univariate Exploration

In [ ]:
# Here explore distributions of individual variables,
# if you see unusual points or outliers, take a deeper look to clean things up 
# and prepare yourself to look at relationships between variables

In [ ]:
# Make sure that, after every plot or related series of plots,
#you include a markdown cell with comments about what you observed and what you plan on investigating next!!!

### Discuss the distributions of your variables of interest. Were there any unusual points? Did you need to perform any transformations?

> asdf

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> asdf

<a id="bivar"></a>
## Bivariate Exploration

In [ ]:
# In this section, investigate relationships between pairs of variables in your data
# Make sure the variables that you cover here have been introduced in some fashion in the previous section (univariate)

### Talk about some of the relationships you observed in this part of the investigation. How did the features of interest vary with other features in the dataset?

> asdf

### Did you observe any interesting relationships between the other features (not the main features of interest)?

> asdf

<a id="multivar"></a>
## Multivariate Exploration

In [ ]:
# create plots of three or more variables to investigate your data further.
# make sure your investigations are justified, and follow from your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your features of interest?

> asdf

### Were there any interesting or surprising interactions between features?

> asdf

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report! XXXXXXX